# 고장난 이미지 찾기

고장난 이미지를 찾기 위해 한번씩 흝어봅니다.


In [25]:
from os import path
import os
from pprint import pprint
from typing import (
    Any, Dict, List, Tuple, NamedTuple, Optional, NewType
)
import typing

from concurrent import futures

from PIL import Image


In [18]:
# typedef
PathStr = NewType('PathStr', str)
Inspection = NamedTuple(
    'Inspection', [('path', PathStr), ('why', 'Exception')]
)


In [32]:
def get_broken(
    top: PathStr, executor: Optional[futures.Executor] = None
) -> List[Inspection]:
    shutdown = False
    if executor is None:
        executor = futures.ThreadPoolExecutor()
        shutdown = True

    tasks = []
    for stem, branches, leaves in os.walk(top):
        for leaf in leaves:
            pathname = path.join(stem, leaf)
            # rel = path.relpath(pathname, top)
            tasks.append(executor.submit(verify_image, pathname))

    broken = []
    results = futures.wait(tasks)
    for faulty in results:
        if faulty:
            pathname, why = faulty
            broken.append(Inspection(path.relpath(pathname, top), why))

    if shutdown:
        executor.shutdown()

    return broken


In [29]:
def image_broken(pathname: PathStr) -> Optional[Tuple[PathStr, Exception]]:
    try:
        with Image.open(pathname) as image:
            image.verify()
    except Exception as why:
        return (pathname, why)
    else:
        return None


In [ ]:
# executor = futures.ThreadPoolExecutor()

train_broken = get_broken('./dataset/training')
val_broken = get_broken('./dataset/validation')

print(len(train_broken))
print(len(val_broken))